In [1]:
from Pytrader_API_V3_02a import Pytrader_API
import pandas as pd
import numpy as np
import pickle
import time
import lightgbm as lgb
import sklearn
MT = Pytrader_API()
brokerInstrumentsLookup = {'EURUSD':'EURUSD', 'GOLD':'XAUUSD', 'DAX':'GER30', 'AUDUSD': 'AUDUSD'}
Connected = MT.Connect(server='127.0.0.1', port=1122, instrument_lookup=brokerInstrumentsLookup)
if MT.Check_connection():
    MT.Set_timeout(120)
    print(MT.Get_broker_server_time())

def log_diff(df):
    df.replace(0, 0.01, inplace=True)
    diffdf = np.log(df).loc[1:,:] - np.log(df.shift(1)).loc[1:,:]
    return diffdf

def minicut(df, train_period=5, pred_period=1):
    '''
    train_period: 用于训练的数据长度，默认前5个交易日
    pred_period: 预测几日后的结果，默认第二天
    '''
    n_features = df.shape[1]
    roll = pd.concat([df.shift(i) for i in range(pred_period,pred_period+train_period-1)], axis=1)
    roll = pd.concat([df, roll], axis=1).dropna()
    return roll.iloc[-1:,:]

2023-10-27 20:54:59


In [2]:
MT.Get_all_open_positions()

,ticket,instrument,order_ticket,position_type,magic_number,volume,open_price,open_time,stop_loss,take_profit,comment,profit,swap,commission
7,26711252,EURUSD,26711252,buy,9826,0.01,1.05688,1698426901,1.05508,1.05717,Test,-0.44,-0.08,0.0
6,26712299,EURUSD,26712299,sell,5498,0.01,1.05615,1698429301,1.05950,1.05553,Test,-0.87,0.01,0.0
5,26712429,EURUSD,26712429,sell,5498,0.01,1.05650,1698429600,1.05985,1.05589,Test,-0.31,0.01,0.0
4,26713093,EURUSD,26713093,sell,5498,0.01,1.05637,1698430501,1.05855,1.05599,Test,-0.52,0.01,0.0
3,26713728,EURUSD,26713728,sell,5498,0.01,1.05640,1698431701,1.06002,1.05572,Test,-0.47,0.01,0.0
2,26714252,EURUSD,26714252,sell,5498,0.01,1.05663,1698433802,1.05979,1.05603,Test,-0.11,0.01,0.0
1,26714404,EURUSD,26714404,sell,5498,0.01,1.05690,1698434700,1.06038,1.05625,Test,0.32,0.01,0.0
0,26714614,EURUSD,26714614,sell,5498,0.01,1.05712,1698436802,1.06124,1.05633,Test,0.66,0.01,0.0


# Test

In [3]:
StaticInfo = MT.Get_static_account_info()
for prop in StaticInfo:
    print("  {}={}".format(prop, StaticInfo[prop]))

  name=Xu Yu Fan
  login=35040601
  currency=AUD
  type=real
  leverage=200
  trade_allowed=True
  limit_orders=200
  margin_call=150.0
  margin_close=50.0
  company=BCR Co Pty Ltd.


In [4]:
DynamicInfo = MT.Get_dynamic_account_info()
for prop in DynamicInfo:
    print("  {}={}".format(prop, DynamicInfo[prop]))

  balance=148.48
  equity=146.73
  profit=-1.75
  margin=50.03
  margin_level=293.26
  margin_free=96.7


In [5]:
InstrumentInfo = MT.Get_instrument_info(instrument='EURUSD')
for prop in InstrumentInfo:
    print("  {}={}".format(prop, InstrumentInfo[prop]))

  instrument=EURUSD
  digits=5
  max_lotsize=20.0
  min_lotsize=0.01
  lot_step=0.01
  point=1e-05
  tick_size=1e-05
  tick_value=1.578906
  swap_long=-5.14
  swap_short=0.73
  stop_level=20


In [6]:
LastTick = MT.Get_last_tick_info(instrument='EURUSD')
for prop in LastTick:
    print("  {}={}".format(prop, LastTick[prop]))

  instrument=EURUSD
  date=1698440099
  ask=1.0567
  bid=1.0566
  last=0.0
  volume=0
  spread=10.0
  date_in_ms=1698440099000


In [7]:
ActualBar = MT.Get_actual_bar_info(
    instrument='EURUSD',
    timeframe=MT.get_timeframe_value('M5'))
for prop in ActualBar:
    print("  {}={}".format(prop, ActualBar[prop]))

  instrument=EURUSD
  date=1698439800
  open=1.0566
  high=1.05666
  low=1.05648
  close=1.05664
  volume=391


In [8]:
LastBars = MT.Get_last_x_bars_from_now(instrument='EURUSD', timeframe=MT.get_timeframe_value('M1'), nbrofbars=1000)
LastBars

array([(1698101340, 1.06684, 1.06686, 1.06683, 1.06686,  27),
       (1698101400, 1.06686, 1.06686, 1.06678, 1.06678,  18),
       (1698101460, 1.06679, 1.06685, 1.06678, 1.06685,  20),
       (1698101520, 1.06683, 1.06685, 1.06683, 1.06685,   9),
       (1698101580, 1.06685, 1.06685, 1.06682, 1.06682,   8),
       (1698101640, 1.06682, 1.06687, 1.06682, 1.06687,  16),
       (1698101700, 1.06687, 1.06687, 1.06682, 1.06683,  12),
       (1698101760, 1.06683, 1.06683, 1.06683, 1.06683,   5),
       (1698101820, 1.06682, 1.06682, 1.06678, 1.06678,   9),
       (1698101880, 1.06677, 1.06678, 1.06675, 1.06677,  21),
       (1698101940, 1.06677, 1.06679, 1.06672, 1.06673,  35),
       (1698102000, 1.06672, 1.06674, 1.06672, 1.06672,  30),
       (1698102060, 1.06673, 1.06673, 1.06672, 1.06672,   9),
       (1698102120, 1.06673, 1.0669 , 1.06672, 1.06688,  42),
       (1698102180, 1.06688, 1.0669 , 1.06688, 1.0669 ,   5),
       (1698102240, 1.06688, 1.06691, 1.06688, 1.06689,  17),
       (

In [9]:
# Send a new order, in this case market order
NewOrder = MT.Open_order(
                instrument='EURUSD',
                ordertype='buy',
                volume=0.01,
                openprice=0.0,
                slippage=10,
                magicnumber=2000,
                stoploss=0.0,
                takeprofit=0.0,
                comment='Test')
if (NewOrder == -1):  # opening order failed
    print(MT.order_error)
    print(MT.order_return_message)

0
Error opening market order


# Demo

In [10]:
while True:
    try:
        tick_time = MT.Get_last_x_bars_from_now(instrument='EURUSD', timeframe=MT.get_timeframe_value('M5'), nbrofbars=10)[-1][0]
        model = pickle.load(open('model/EURUSD.pickle', 'rb')).booster_
        while True:
            LastBars = MT.Get_last_x_bars_from_now(instrument='EURUSD', timeframe=MT.get_timeframe_value('M5'), nbrofbars=100)
            actual_bar_info = MT.Get_actual_bar_info(instrument='EURUSD', timeframe=MT.get_timeframe_value('M5'))
            # if LastBars[-1][0] <= tick_time:
            #     for index, row in MT.Get_all_open_positions().iterrows():
            #         if row['position_type']=='buy' and (row['open_price'] + 5/10000 < actual_bar_info['close'] or row['open_price'] - 10/10000 > actual_bar_info['close']):
            #             MT.Close_position_by_ticket(ticket=row['ticket'])
            #         elif row['position_type']=='sell' and (row['open_price'] - 5/10000 > actual_bar_info['close'] or row['open_price'] + 10/10000 < actual_bar_info['close']):
            #             MT.Close_position_by_ticket(ticket=row['ticket'])
            if LastBars[-1][0] > tick_time and MT.Get_dynamic_account_info()['margin_free'] > 60:
                tick_time = LastBars[-1][0]
                X = np.ascontiguousarray(minicut(log_diff(pd.DataFrame(LastBars).iloc[:,1:]),10,1))
                pchg = (np.exp(model.predict(X)[0])-1) * 100000
                print(time.ctime(), pchg, end=' ')
                LastTick = MT.Get_last_tick_info(instrument='EURUSD')
                sl = round(LastBars[-1][-2]*(1-5*pchg/10000), 5)
                tk = round(LastBars[-1][-2]*(1+pchg/10000), 5)
                if  pchg > 1 and tk - 10/100000 > LastTick['ask']:
                    NewOrder = MT.Open_order(
                                instrument='EURUSD',
                                ordertype='buy',
                                volume=0.01,
                                openprice=0.0,
                                slippage=10,
                                magicnumber=9826,
                                stoploss=sl,
                                takeprofit=tk,
                                comment='Test')
                    if NewOrder == -1:
                        raise Exception
                    print(NewOrder, 'buy 0.01', end=' ')
                    # MT.Set_sl_and_tp_for_position(ticket=NewOrder, stoploss=round(actual_bar_info['close']*(1-2*pchg/10000), 5), takeprofit=0.0)
                    # MT.Set_sl_and_tp_for_position(ticket=NewOrder, stoploss=0.0, takeprofit=round(actual_bar_info['close']*(1+pchg/10000), 5))
                elif pchg < -1 and tk + 10/100000 < LastTick['bid']:
                    NewOrder = MT.Open_order(
                                instrument='EURUSD',
                                ordertype='sell',
                                volume=0.01,
                                openprice=0.0,
                                slippage=10,
                                magicnumber=5498,
                                stoploss=sl,
                                takeprofit=tk,
                                comment='Test')
                    if NewOrder == -1:
                        raise Exception
                    print(NewOrder, 'sell 0.01', end=' ')
                    # MT.Set_sl_and_tp_for_position(ticket=NewOrder, stoploss=round(actual_bar_info['close']*(1-2*pchg/10000), 5), takeprofit=0.0)
                    # MT.Set_sl_and_tp_for_position(ticket=NewOrder, stoploss=0.0, takeprofit=round(actual_bar_info['close']*(1+pchg/10000), 5))
                print()
    except:
        continue

Mon Oct 30 08:04:59 2023 -1.7594113833840908 26715024 sell 0.01 
Mon Oct 30 08:06:02 2023 -0.380425896251424 
Mon Oct 30 08:12:49 2023 -0.8103008337645434 
Mon Oct 30 08:16:08 2023 0.8916502935329973 
Mon Oct 30 08:21:19 2023 -0.23655409606693212 
Mon Oct 30 08:26:01 2023 -4.761657484131376 26715044 sell 0.01 
Mon Oct 30 08:31:45 2023 -1.1233689136691538 
Mon Oct 30 08:36:02 2023 -0.17724809489028104 
Mon Oct 30 08:41:04 2023 -0.21546793116389296 
Mon Oct 30 08:46:33 2023 0.6040492346759407 
Mon Oct 30 08:51:00 2023 0.8037169014318835 
Mon Oct 30 08:56:25 2023 -1.2191325199584213 
Mon Oct 30 09:00:58 2023 -0.4507825624888717 
Mon Oct 30 09:06:01 2023 0.5463203337940925 
Mon Oct 30 09:11:01 2023 0.55141904038436 
Mon Oct 30 09:15:58 2023 0.826749967353102 
Mon Oct 30 09:21:35 2023 1.2904490408693903 
Mon Oct 30 09:26:33 2023 -5.418933832501693 26715514 sell 0.01 
Mon Oct 30 09:30:58 2023 0.4633321425639991 
Mon Oct 30 09:36:09 2023 1.7576923293072255 26715703 buy 0.01 
Mon Oct 30 09:41: